#SS_DSVDD

#IMPORTED FILES AND NECESSARY MODULES

In [80]:
!pip install barbar
from google.colab import drive
import torch
import numpy as np
from torchvision.datasets import MNIST
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Subset
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import roc_auc_score
from barbar import Bar
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# UTILITY CLASSES

In [81]:

class MyMNIST(MNIST):
    def __init__(self, *args, **kwargs):
        super(MyMNIST, self).__init__(*args, **kwargs)

        self.semi_targets = torch.zeros_like(self.targets)

    def __getitem__(self, index):
        img, target, semi_target = self.data[index], int(self.targets[index]), int(self.semi_targets[index])
        img = Image.fromarray(img.numpy(), mode='L')
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, semi_target, index
class network(nn.Module):
    def __init__(self, z_dim=32):
        super(network, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 7 * 7, z_dim, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        return self.fc1(x)


class encoder(nn.Module):
    def __init__(self, z_dim=32):
        super(autoencoder, self).__init__()
        self.z_dim = z_dim
        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 7 * 7, z_dim, bias=False)

        self.deconv1 = nn.ConvTranspose2d(2, 4, 5, bias=False, padding=2)
        self.bn3 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(4, 8, 5, bias=False, padding=3)
        self.bn4 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(8, 1, 5, bias=False, padding=2)
        
    def encode(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        return self.fc1(x)
   
    def decode(self, x):
        x = x.view(x.size(0), int(self.z_dim / 16), 4, 4)
        x = F.interpolate(F.leaky_relu(x), scale_factor=2)
        x = self.deconv1(x)
        x = F.interpolate(F.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.deconv3(x)
        return torch.sigmoid(x)
        

    def forward(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        return x_hat
class TrainerDeepSAD:
    def __init__(self, args, data, device):
        self.args = args
        self.train_loader, self.test_loader = data
        self.device = device
    

    def pretrain(self):
        ae = autoencoder(self.args.latent_dim).to(self.device)
        ae.apply(weights_init_normal)
        optimizer = optim.Adam(ae.parameters(), lr=self.args.lr_ae,
                               weight_decay=self.args.weight_decay_ae)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                    milestones=self.args.lr_milestones, gamma=0.1)
        
        ae.train()
        for epoch in range(self.args.num_epochs_ae):
            total_loss = 0
            for x, _, _, _ in Bar(self.train_loader):
                x = x.float().to(self.device)
                
                optimizer.zero_grad()
                x_hat = ae(x)
                reconst_loss = torch.mean(torch.sum((x_hat - x) ** 2, dim=tuple(range(1, x_hat.dim()))))
                reconst_loss.backward()
                optimizer.step()
                
                total_loss += reconst_loss.item()
            scheduler.step()
            print('Pretraining Autoencoder... Epoch: {}, Loss: {:.3f}'.format(
                   epoch, total_loss/len(self.train_loader)))
        self.save_weights_for_DeepSAD(ae, self.train_loader) 
    

    def save_weights_for_DeepSAD(self, model, dataloader):
        c = self.set_c(model, dataloader)
        net = network(self.args.latent_dim).to(self.device)
        state_dict = model.state_dict()
        net.load_state_dict(state_dict, strict=False)
        torch.save({'center': c.cpu().data.numpy().tolist(),
                    'net_dict': net.state_dict()}, '/content/drive/MyDrive/pretrained_parameters.pth')
    

    def set_c(self, model, dataloader, eps=0.1):
        model.eval()
        z_ = []
        with torch.no_grad():
            for x, _, _, _ in dataloader:
                x = x.float().to(self.device)
                z = model.encode(x)
                z_.append(z.detach())
        z_ = torch.cat(z_)
        c = torch.mean(z_, dim=0)
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
        return c


    def train(self):
        net = network().to(self.device)
        eta=1
        eps=1e-6
        if self.args.pretrain==True:
            state_dict = torch.load('/content/drive/MyDrive/pretrained_parameters.pth')
            net.load_state_dict(state_dict['net_dict'])
            c = torch.Tensor(state_dict['center']).to(self.device)
        else:
            net.apply(weights_init_normal)
            c = torch.randn(self.args.latent_dim).to(self.device)
        
        optimizer = optim.Adam(net.parameters(), lr=self.args.lr,
                               weight_decay=self.args.weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                    milestones=self.args.lr_milestones, gamma=0.1)

        net.train()
        for epoch in range(self.args.num_epochs):
            total_loss = 0
            for x, _, sy, _ in Bar(self.train_loader):
                x = x.float().to(self.device)
                sy=sy.float().to(self.device)
                optimizer.zero_grad()
                z = net(x)
                dist = torch.sum((z-c) ** 2, dim=1)
                losses = torch.where(sy == 0, dist, eta * ((dist + eps) ** sy.float()))
                loss = torch.mean(losses)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            scheduler.step()
            print('Training Deep SAD... Epoch: {}, Loss: {:.3f}'.format(
                   epoch, total_loss/len(self.train_loader)))
        self.net = net
        self.c = c
class Args:
  def __init__(self, nclass):
    self.num_epochs=20
    self.num_epochs_ae=20
    self.patience=50
    self.lr=1e-4
    self.weight_decay=0.5e-6
    self.weight_decay_ae=0.5e-3
    self.lr_ae=1e-4
    self.lr_milestones=[50]
    self.batch_size=200
    self.pretrain=True
    self.latent_dim=32
    self.normal_class=nclass
    self.ratio_known_outlier=0.01
    self.ratio_known_normal=0.0
    self.ratio_pollution=0.1
    self.known_outlier_class=(nclass+1)%10
    self.n_known_outlier_classes=1

# UTILITY FUNCTIONS

In [82]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1 and classname != 'Conv':
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
def create_semisupervised_setting(labels, normal_classes, outlier_classes, known_outlier_classes,ratio_known_normal, ratio_known_outlier, ratio_pollution):
    idx_normal = np.argwhere(np.isin(labels, normal_classes)).flatten()
    idx_outlier = np.argwhere(np.isin(labels, outlier_classes)).flatten()
    idx_known_outlier_candidates = np.argwhere(np.isin(labels, known_outlier_classes)).flatten()

    n_normal = len(idx_normal)

    a = np.array([[1, 1, 0, 0],
                  [(1-ratio_known_normal), -ratio_known_normal, -ratio_known_normal, -ratio_known_normal],
                  [-ratio_known_outlier, -ratio_known_outlier, -ratio_known_outlier, (1-ratio_known_outlier)],
                  [0, -ratio_pollution, (1-ratio_pollution), 0]])
    b = np.array([n_normal, 0, 0, 0])
    x = np.linalg.solve(a, b)

    n_known_normal = int(x[0])
    n_unlabeled_normal = int(x[1])
    n_unlabeled_outlier = int(x[2])
    n_known_outlier = int(x[3])

    perm_normal = np.random.permutation(n_normal)
    perm_outlier = np.random.permutation(len(idx_outlier))
    perm_known_outlier = np.random.permutation(len(idx_known_outlier_candidates))

    idx_known_normal = idx_normal[perm_normal[:n_known_normal]].tolist()
    idx_unlabeled_normal = idx_normal[perm_normal[n_known_normal:n_known_normal+n_unlabeled_normal]].tolist()
    idx_unlabeled_outlier = idx_outlier[perm_outlier[:n_unlabeled_outlier]].tolist()
    idx_known_outlier = idx_known_outlier_candidates[perm_known_outlier[:n_known_outlier]].tolist()

    labels_known_normal = labels[idx_known_normal].tolist()
    labels_unlabeled_normal = labels[idx_unlabeled_normal].tolist()
    labels_unlabeled_outlier = labels[idx_unlabeled_outlier].tolist()
    labels_known_outlier = labels[idx_known_outlier].tolist()

    semi_labels_known_normal = np.ones(n_known_normal).astype(np.int32).tolist()
    semi_labels_unlabeled_normal = np.zeros(n_unlabeled_normal).astype(np.int32).tolist()
    semi_labels_unlabeled_outlier = np.zeros(n_unlabeled_outlier).astype(np.int32).tolist()
    semi_labels_known_outlier = (-np.ones(n_known_outlier).astype(np.int32)).tolist()

    list_idx = idx_known_normal + idx_unlabeled_normal + idx_unlabeled_outlier + idx_known_outlier
    list_labels = labels_known_normal + labels_unlabeled_normal + labels_unlabeled_outlier + labels_known_outlier
    list_semi_labels = (semi_labels_known_normal + semi_labels_unlabeled_normal + semi_labels_unlabeled_outlier
                        + semi_labels_known_outlier)

    return list_idx, list_labels, list_semi_labels
def get_mnist(args, data_dir='/content/drive/MyDrive/Data'):
    n_classes = 2
    normal_classes = tuple([args.normal_class])
    outlier_classes = list(range(0, 10))
    outlier_classes.remove(args.normal_class)
    outlier_classes = tuple(outlier_classes)

    if args.n_known_outlier_classes == 0:
        known_outlier_classes = ()
    elif args.n_known_outlier_classes == 1:
        known_outlier_classes = tuple([args.known_outlier_class])
    else:
        known_outlier_classes = tuple(random.sample(outlier_classes, args.n_known_outlier_classes))
    
    transform = transforms.ToTensor()
    target_transform = transforms.Lambda(lambda x: int(x in outlier_classes))
    
    train = DataSet(root=data_dir, train=True,transform=transform, target_transform=target_transform,download=True)
    idx, _, semi_targets = create_semisupervised_setting(train.targets.cpu().data.numpy(), normal_classes,
                                                             outlier_classes, known_outlier_classes,
                                                             args.ratio_known_normal, args.ratio_known_outlier, args.ratio_pollution)
    train.semi_targets[idx] = torch.tensor(semi_targets)
    train = Subset(train, idx)          
    dataloader_train = DataLoader(train, batch_size=args.batch_size, 
                                  shuffle=True, num_workers=0)
    
    test = DataSet(root=data_dir, train=False, transform=transform, target_transform=target_transform,
                                download=True)
    dataloader_test = DataLoader(test, batch_size=args.batch_size, 
                                  shuffle=True, num_workers=0)
    return dataloader_train, dataloader_test

# OUTPUT

In [83]:
nscores=[]
roc=[]
mean=[]
for i in range(10):
  args = Args(i)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  data = get_mnist(args)
  deep_SAD = TrainerDeepSAD(args, data, device)
  if args.pretrain:
      deep_SAD.pretrain()
  deep_SAD.train()
  print("test for calss  ",i,".....")
  net=deep_SAD.net
  c=deep_SAD.c
  dataloader=data[1]
  nscores.append(roc_auc_score(labels, scores)*100)
  scores = []
  labels = []
  epoch_loss=0
  nb=0
  eta=1.0
  eps=1e-6
  net.eval()
  with torch.no_grad():
      for x, y, sy,_ in dataloader:
          x = x.float().to(device)
          y = y.float().to(device)
          sy= sy.float().to(device)
          z = net(x)
          dist = torch.sum((z-c) ** 2, dim=1)
          losses = torch.where(sy == 0, dist, eta * ((dist + eps) ** sy.float()))
          loss = torch.mean(losses)
          score=dist
          scores.append(score.detach().cpu())
          labels.append(y.cpu())
          epoch_loss+=loss
          nb+=1
  labels, scores = torch.cat(labels).numpy(), torch.cat(scores).numpy()
  roc.append(roc_auc_score(labels, scores)*100)
  mean.append(epoch_loss/nb)


6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 0, Loss: 170.151
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 1, Loss: 135.092
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 2, Loss: 109.937
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 3, Loss: 92.076
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 4, Loss: 79.379
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 5, Loss: 70.294
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 6, Loss: 63.669
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 7, Loss: 58.564
6647/6647: [===============================>] - ETA 0.2s
Pretraining Autoencoder... Epoch: 8, Loss: 54.459
6647/6647: [======================

In [84]:
print('#==============#==============#==============#==============#')
print('|     CLASS    |   AUC SCORE  |   MEAN LOSS  |  ROC SCORE   |')
for i in range(10):
  print('|--------------|--------------|--------------|--------------|')
  print('|    ',i,'       |    {:.2f}'.format(roc[i]),'    |      {:.2f}'.format(mean[i]),'   |     {:.2f}    |'.format(nscores[i]))
print('#==============#==============#==============#==============#')

#==============#==============#==============#==============#
|     CLASS    |   AUC SCORE  |   MEAN LOSS  |  ROC SCORE   |
|--------------|--------------|--------------|--------------|
|     0        |    95.56     |      0.12    |     98.90    |
|--------------|--------------|--------------|--------------|
|     1        |    98.44     |      0.14    |     95.56    |
|--------------|--------------|--------------|--------------|
|     2        |    87.37     |      0.12    |     98.44    |
|--------------|--------------|--------------|--------------|
|     3        |    94.80     |      0.14    |     87.37    |
|--------------|--------------|--------------|--------------|
|     4        |    91.76     |      0.12    |     94.80    |
|--------------|--------------|--------------|--------------|
|     5        |    83.35     |      0.09    |     91.76    |
|--------------|--------------|--------------|--------------|
|     6        |    97.81     |      0.14    |     83.35    |
|-------